In [1]:
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 877.8 kB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0


# **Importing Libraries**
We begin by importing the necessary libraries:

**langchain:** Used to manage prompt templates, chains, and memory for conversational contexts.

**OpenAI:** Provides the language model for response generation.

In [1]:
%%writefile requirements.txt
langchain
langchain-community
langchain-openai
pypdf
langchain-chroma
gradio

Writing requirements.txt


In [2]:
!pip install -q -r  requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00

In [3]:
from langchain_openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from google.colab import userdata
import os
import pandas as pd
import numpy as np

# **Setting Up Environment Variables**
To interact with the OpenAI API, you need an API key. The code snippet below fetches the API key stored as an environment variable. Ensure the variable OPENAI_API_KEY is set in your environment for the chatbot to function correctly.

In [6]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY1')

# **Indexing**
We start by preparing our documents for retrieval using embeddings and vector storage.

## **Load**
In this step, we load news data. The data used here is from the reuters-21578 dataset.

You can upload this or other documents to provide a robust foundation for the chatbot's responses.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
file_path = '/content/drive/MyDrive/topics_classification_dataset.csv'
df = pd.read_csv(file_path)
df

,LEWISSPLIT,Text,Topics
0,TRAIN,JAGUAR SEES STRONG GROWTH IN NEW MODEL SALES J...,earn
1,TRAIN,NORD RESOURCES CORP <NRD> 4TH QTR NET Shr 19 c...,earn
2,TRAIN,FIVE GROUPS APPLY TO BUY FRENCH TELEPHONE GROU...,acq
3,TRAIN,BLIZZARD CLOSES BOSPHORUS Blizzard conditions ...,ship
4,TRAIN,JAPAN FUND <JPN> SEEKERS CONFIDENT OF FINANCIN...,acq
...,...,...,...
7052,TRAIN,BAKER INTERNATIONAL CORP SUES HUGHES TOOL SEEK...,acq
7053,TRAIN,USAIR GROUP REJECTS TRANS WORLD AIRLINES TAKEO...,acq
7054,TRAIN,BAKER <BKO> SUES TO FORCE HUGHES <HT> MERGER B...,acq
7055,TRAIN,SPAIN DEREGULATES BANK DEPOSIT INTEREST RATES ...,interest


In [9]:
column_name = 'Text'
text_data = df[column_name].dropna().tolist()

## **Store**
We create embeddings of our text chunks and store them in a vector database. This allows us to search for similar content efficiently.

In [10]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_texts(text_data, embeddings)

# **Retrieval and Generation**
With the indexed documents, we can now retrieve relevant information and generate responses based on user questions.

## **Retrieve**
We set up a retriever to find content related to a user's query based on similarity with stored embeddings.

In [11]:
retriever = db.as_retriever(search_type= "similarity")

In [12]:
question = retriever.invoke("Who did Irving Bank Corp acquire a division from?")

In [13]:
text_data[0]

'JAGUAR SEES STRONG GROWTH IN NEW MODEL SALES Jaguar Plc <JAGR.L> is about to sell its\nnew XJ-6 model on the U.S. And Japanese markets and expects a\nstrong reception based on its success in the U.K., Chairman Sir\nJohn Egan told a news conference.\n    Commenting on an 11 pct growth in 1986 group turnover to\n830.4 mln stg and pre-tax profits at 120.8 mln stg, slightly\nbelow 1985\'s 121.3 mln, Egan said Jaguar aimed at an average\nprofit growth of 15 pct per year. However, the introduction of\nthe new model had kept this year\'s pre-tax profit down.\n    Jaguar starts selling XJ-6 in the U.S. In May and plans to\nsell 25,000 of its total 47,000 production there in 1987.\n    U.S. Sales now account for 65 pct of total turnover,\nfinance director John Edwards said.\n    A U.S. Price for the car has not been set yet, but Edwards\nsaid the relatively high car prices in dollars of West German\ncompetitors offered an "umbrella" for Jaguar. He added the XJ-6\nhad also to compete with U.S. 

In [14]:
for i in range(len(question)):
  print(f"the {i+1}th similar content :\n \n {question[i].page_content}\n \n")

the 1th similar content :
 
 IRVING TRUST <V> BUYS GULF/WESTERN <GW> UNIT Irving Bank Corp said it bought the
factoring division of Associates Commercial Corp, a unit of
Gulf and Western Co Inc's Associates Corp of North America.
    The terms of the previously announced deal were not
disclosed.
    It said the assets were transferred to Irving Commercial
Corp.

 Reuter

 

the 2th similar content :
 
 BANK OF NEW YORK REAFFIRMS TERMS FOR IRVING <V> The Bank of New York Corp <BK> said it
reaffirmed the terms of its offer for Irving Bank Corp despite
the drop in the Bank of New York's share price to 30-1/8, a
Bank of New York spokesman said.
    "The offer still stands, we have not changed our offer," a
Bank of New York spokesman said.
    Irving would not comment on how the drop in the market
affects its position on the bid or whether it would buy back
any of its own shares.
    Earlier this month, Irving rejected the bid as inadequate
and said it wanted to retain its independence.
  

## **Multi Query**
we create multiple versions of a question to capture different perspectives or possible interpretations. This improves retrieval diversity.

In [15]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [16]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [17]:
generate_queries.invoke("Who did Irving Bank Corp acquire a division from?")

['1. From which company did Irving Bank Corp acquire a division?',
 '2. What company did Irving Bank Corp purchase a division from?',
 '3. Which division was acquired by Irving Bank Corp from another company?',
 '4. Whose division did Irving Bank Corp acquire?',
 '5. What was the name of the company from which Irving Bank Corp acquired a division?']

In [18]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "Who did Irving Bank Corp acquire a division from?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})

<ipython-input-18-0dadb7109b1a>:9: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


In [19]:
for i in range(len(docs)):
  print(f"{i+1}. {docs[1].page_content}\n \n")

1. BANK OF NEW YORK REAFFIRMS TERMS FOR IRVING <V> The Bank of New York Corp <BK> said it
reaffirmed the terms of its offer for Irving Bank Corp despite
the drop in the Bank of New York's share price to 30-1/8, a
Bank of New York spokesman said.
    "The offer still stands, we have not changed our offer," a
Bank of New York spokesman said.
    Irving would not comment on how the drop in the market
affects its position on the bid or whether it would buy back
any of its own shares.
    Earlier this month, Irving rejected the bid as inadequate
and said it wanted to retain its independence.
    In late September, Bank of New York offered 80 dlrs per
share in cash for 47.4 pct of Irving. For the remaining 52.6
pct, it offered an exchange of 1.9 shares of its shares for one
Irving share.
    At that time, the stock purchase portion was worth close to
80 dlrs per share, but now that portion is worth 53 dlrs per
share for a net price of 68 dlrs, one analyst said.
    According to the prospectu

## **Generate**
We process the retrieved information to generate a response using a language model, enhancing the chatbot's ability to answer complex queries.

In [20]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser

# RAG template
template = """Use the given context and conversation history to directly answer the question concisely:

Context:
{context}

Chat history:
{chat_history}

New human question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Initialize model and memory
llm = ChatOpenAI(temperature=1)
memory = ConversationBufferWindowMemory(k = 3, memory_key="chat_history")

# Define RAG pipeline without memory directly
final_rag_chain = (
    {"context": itemgetter("context"),
     "question": itemgetter("question"),
     "chat_history": itemgetter("chat_history")}  # Include chat_history as a key here
    | prompt
    | llm
    | StrOutputParser()
)

# Retrieve the chat history from memory and include it in the input
def invoke_with_memory(question, retrieval_chain):
    # Load current chat history from memory
    chat_history = memory.load_memory_variables({}).get("chat_history", "")
    result = final_rag_chain.invoke({
        "question": question,
        "context": retrieval_chain,  # Assuming retrieval_chain is defined elsewhere
        "chat_history": chat_history,
    })
    # Update memory with the new interaction
    memory.save_context({"question": question}, {"answer": result})
    return result

<ipython-input-20-ab79890371de>:24: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k = 3, memory_key="chat_history")


## **UI**
Finally, we build a simple interface with Gradio to interact with the news chatbot. Users can type questions, and the chatbot will respond with relevant information.

In [21]:
import gradio as gr

chat_history = []

def news_chatbot(query):
    global chat_history

    response = invoke_with_memory(query, retrieval_chain)
    response = response.replace("Based on the context and previous conversation, ", "")
    response = response.replace("Based on the conversation and context provided, ", "")
    response = response.replace("Based on our previous conversation,", "")
    response = response.replace("Based on the context provided and our previous conversation,", "")


    chat_history.append((query, response))

    return chat_history

def reset_conversation():
    global chat_history
    chat_history = []
    memory.aclear()
    return chat_history

with gr.Blocks() as interface:
    gr.Markdown("News chatbot")
    gr.Markdown("Ask me any News question, and I'll try to provide helpful information based on the provided data.")

    chatbot = gr.Chatbot()
    query = gr.Textbox(label="Your Question", placeholder="Type your medical question here...")

    submit_button = gr.Button("Get Answer")
    reset_button = gr.Button("Start New Conversation")

    submit_button.click(fn=news_chatbot, inputs=query, outputs=chatbot)
    reset_button.click(fn=reset_conversation, inputs=None, outputs=chatbot)

    submit_button.click(lambda: "", None, query)

interface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://31615f4462efcf2e33.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
